In [5]:
import numpy as np 
import pandas as pd
from subprocess import check_output

이 커널은 특히 Keras를 사용하여 CNN 구축을 실험하려는 초보자를 위한 것입니다. 
이 커널을 사용하면 좋은 점수를 얻고 케라스를 배울 수 있습니다. Keras는 모델을 초기화하고 원하는 레이어를 계속 쌓을 수있는 간단한 프레임 워크입니다. 
심층 신경망을 매우 쉽게 구축 할 수 있습니다.

In [6]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from os.path import join as opj
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pylab
plt.rcParams['figure.figsize'] = 10, 10
%matplotlib inline

In [7]:
# json파일을 불러오기 위해 pd.read_json을 사용한다.
#Load the data.
train = pd.read_json("./input/train.json")

print(train.head())

         id                                             band_1  \
0  dfd5f913  [-27.878360999999998, -27.15416, -28.668615, -...   
1  e25388fd  [-12.242375, -14.920304999999999, -14.920363, ...   
2  58b2aaa0  [-24.603676, -24.603714, -24.871029, -23.15277...   
3  4cfc3a18  [-22.454607, -23.082819, -23.998013, -23.99805...   
4  271f93f4  [-26.006956, -23.164886, -23.164886, -26.89116...   

                                              band_2 inc_angle  is_iceberg  
0  [-27.154118, -29.537888, -31.0306, -32.190483,...   43.9239           0  
1  [-31.506321, -27.984554, -26.645678, -23.76760...   38.1562           0  
2  [-24.870956, -24.092632, -20.653963, -19.41104...   45.2859           1  
3  [-27.889421, -27.519794, -27.165262, -29.10350...   43.8306           0  
4  [-27.206915, -30.259186, -30.259186, -23.16495...   35.6256           0  


In [8]:
test = pd.read_json("./input/test.json")

주어진 이미지가 빙하인지 배인지 구분하는 문제.

feature로는 band_1, band_2, inc_angle이 있음.

Sentinental 1 위성은 지구를 공전하며 위성레이더를 통해 지구 상의 여러 물체들을 감지함.
위성 레이더는 객체에서 신호를 반사하고 에코를 기록한 다음 해당 데이터가 이미지로 변환됨.
훈련 데이터에서 주어지는 band_1은 HH방식(수평 전송/수평 수신)으로 얻은 이미지
band_2는 HV방식(수평 전송/수직 수신)으로 얻은 이미지. 

inc_angle 은 입사각. is_iceberg는 우리가 구해야 하는 레이블. 

X_band_1과 X_band_2의 평균을 새로운 채널로 하여 3개의 채널을 가진 새로운 훈련 데이터를 만든다. 



In [44]:
# training data 를 만듬
# HH, HV, avg를 가진 3개의 밴드를 만듬.
# band_1과 band_2를 75 x 75로 바꾸고, 2D CNN 학습을 위하여 [N, height, width, channel]의 차원을 가지게 바꿈.

X_band_1 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_1"]])

X_band_2 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_2"]])

print(X_band_1.shape)
X_train = np.concatenate([X_band_1[:, :, :, np.newaxis], 
                         X_band_2[:, :, :, np.newaxis],
                         ((X_band_1 + X_band_2)/2)[:, :, :, np.newaxis]], axis=-1)


(1604, 75, 75)


In [10]:

#Take a look at a iceberg
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)

def plotmy3d(c, name):
    data = [
        go.Surface(
            z=c
        )
    ]
    layout = go.Layout(
        title=name,
        autosize=False,
        width=700,
        height=700,
        margin=dict(
            l=65,
            r=50,
            b=65,
            t=90
        )
    )
    fig = go.Figure(data=data, layout=layout)
    
    py.iplot(fig)
    
plotmy3d(X_band_1[12, :, :], 'iceberg')

우리가 가지고 있는 멋진 iceberg입니다. 빙산의 모양은 여기에 표시된 것처럼 산과 같습니다. 
그러나 선박의 모양은 점과 같을 것이고, 가늘고 긴 점과 같을 수 있습니다. 

Keras를 사용하여 CNN 구축으로 돌아가십시오. 다른 프레임 워크보다 훨씬 나은 프레임 워크. 당신은 확실히 즐거울 것입니다.

우리가 가지고 있는 멋진 빙산입니다.
빙산의 모양은 여기에 표시된 것처럼 산과 같습니다. 
그러나 선박의 모양은 점과 같을 것이고, 길쭉한 점과 같을 수 있습니다. 
여기에서 구조적 차이가 발생하고 CNN을 사용하여 이러한 차이를 활용할 수 있습니다.

Keras를 사용하여 CNN 구축으로 돌아가십시오. 다른 프레임 워크보다 훨씬 나은 프레임 워크이고 당신은 확실히 즐길 수 있을 것입니다.

In [11]:
#import  Keras
from matplotlib import pyplot
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, Activation
from keras.layers import GlobalMaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras import initializers
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping

In [26]:
def getModel():
    gmodel = Sequential()
    
    # Conv Layer 1
    gmodel.add(Conv2D(64, (3, 3), activation='relu', input_shape=(75, 75, 3)))
    gmodel.add(MaxPooling2D(3, strides=2))
    gmodel.add(Dropout(0.2))
    
    # Conv Layer 2
    gmodel.add(Conv2D(128, (3, 3), activation='relu'))
    gmodel.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    gmodel.add(Dropout(0.2))

    # Conv Layer 3
    
    gmodel.add(Conv2D(128, kernel_size=(3,3), activation='relu'))
    gmodel.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
    gmodel.add(Dropout(0.2))
    
    #Conv Layer 4
    gmodel.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
    gmodel.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    gmodel.add(Dropout(0.2))
    
    #Flatten the data for upcoming dense layers
    gmodel.add(Flatten())
    
    #Dense Layers
    
    gmodel.add(Dense(256))
    gmodel.add(Activation('relu'))
    gmodel.add(Dropout(0.2))
    
    #Sigmoid Layer
    gmodel.add(Dense(1))
    gmodel.add(Activation('sigmoid'))
    
    mypotim = Adam(lr=0.001, beta_1=0.9, beta_2 = 0.999, epsilon=1e-08, decay=0.0)
    gmodel.compile(loss='binary_crossentropy', optimizer=mypotim, metrics=['accuracy'])
    gmodel.summary()
    return gmodel

def get_callbacks(filepath, patience=2):
    es = EarlyStopping('val_loss', patience=patience, mode="min")
    msave=ModelCheckpoint(filepath, save_best_only=True)
    return [es, msave]

file_path=".model_weights.hdf5"
callbacks = get_callbacks(filepath=file_path, patience=5)
    

In [27]:

target_train = train['is_iceberg']
X_train_cv, X_valid, y_train_cv, y_valid= train_test_split(X_train, target_train, random_state=16, train_size=0.75)

In [29]:
#without denoising, core features

import os
gmodel = getModel()
gmodel.fit(X_train_cv, y_train_cv, 
          batch_size=32,
          epochs=30,
          verbose=1,
          validation_data=(X_valid, y_valid),
          callbacks=callbacks)

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 73, 73, 64)        1792      
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 36, 36, 64)        0         
_________________________________________________________________
dropout_14 (Dropout)         (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 17, 17, 128)       0         
_________________________________________________________________
dropout_15 (Dropout)         (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 15, 15, 128)      

In [48]:

target_train = train['is_iceberg']
X_train_cv, X_valid, y_train_cv, y_valid = train_test_split(X_train, target_train, random_state=16, train_size=0.75)

In [49]:
print(file_path)

.model_weights.hdf5


In [50]:
gmodel.load_weights(filepath=file_path)

score = gmodel.evaluate(X_valid, y_valid, verbose=1)

TypeError: load_weights() missing 1 required positional argument: 'filepath'

In [45]:
X_band_test_1 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_1"]])
X_band_test_2 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_2"]])

X_test = np.concatenate([X_band_test_1[:, :, :, np.newaxis], X_band_test_2[:, :, :, np.newaxis], 
                         ((X_band_test_1+X_band_test_2)/2)[:, :, :, np.newaxis]], axis=-1)

predicted_test = gmodel.predict_proba(X_test)

In [9]:

submission = pd.DataFrame()

submission['id'] = test['id']

submission['is_iceberg'] = predicted_test.reshape((predicted_test.shape[0]))
submission.to_csv('sub.csv', index=False)

NameError: name 'predicted_test' is not defined

결론
점수를 높이기 위해 Speckle 필터링, Indicence angle normalization 및 기타 전처리를 시도했지만 작동하지 않는 것 같습니다. 당신은 시도하고 볼 수 있지만 나에게 그들은 좋은 결과를주지 않습니다.

이 커널을 사용하여 10 위를 차지할 수 없으므로 여기에 아름다운 정보가 있습니다. 테스트 데이터 세트에는 8000 개의 이미지가 포함되어 있습니다. 의사 라벨링을 수행하여 예측을 높일 수 있습니다. 여기에 관련 기사가 있습니다 : https://towardsdatascience.com/simple-explanation-of-semi-supervised-learning-and-pseudo-labeling-c2218e8c769b

이 커널이 마음에 들면 Upvote.